# Cálculo de otros indicadores simples

In [15]:
from scripts.green_land import calculate_green_spaces
from scripts.count_points import count_points
from scripts.path_finders import get_division_path, get_context_shp_path, get_transport_shp_path
from scripts.create_indicators import add_indicator, get_or_create_indicators_df
import shapefile
from shapely.geometry import MultiPoint
from scripts.geo_utils import iter_shp_as_shapely

In [2]:
radio_shp = get_division_path("radios_censo_2010")
fracc_shp = get_division_path("fracciones_caba_censo_2010")
#barrio_shp = get_division_path("barrios_censo_2010")
comuna_shp = get_division_path("comunas_caba_censo_2010")

## 1. Incidencia de espacios verdes

Calculamos la incidencia de espacios verdes en cada radio censal.

In [6]:
green_privados_shp = get_context_shp_path("espacios-verdes-privados")
green_publicos_shp = get_context_shp_path("espacios-verdes-publicos")

In [7]:
radios_green = calculate_green_spaces(radio_shp, [green_privados_shp, green_publicos_shp])

In [8]:
radios_green["1_13_14"]

0.5412636715077378

Agregamos el indicador a la base

In [9]:
df = add_indicator("RADIO", "esp_verde", radios_green)

## 2. Cantidad de "puntos de interés"

Contamos la cantidad de hospitales y establecimientos educativos por cada radio censal.

In [3]:
hospitales_shp = get_context_shp_path("hospitales")
educ_privados_shp = get_context_shp_path("Establecimientos Educativos Privados")
educ_publicos_shp = get_context_shp_path("Establecimientos Educativos Publicos")

In [4]:
hospitales = count_points(hospitales_shp, radio_shp)

In [5]:
print sum(hospitales.values()), "hospitales contados"

36 hospitales contados


In [6]:
educ_privados = count_points(educ_privados_shp, radio_shp)
educ_publicos = count_points(educ_publicos_shp, radio_shp)

In [7]:
print sum(educ_privados.values()), "establecimientos educativos privados contados"
print sum(educ_publicos.values()), "establecimientos educativos públicos contados"

982 establecimientos educativos privados contados
955 establecimientos educativos públicos contados


Agregamos los indicadores a la base

In [8]:
df = add_indicator("RADIO", "hospitales", hospitales)
df = add_indicator("RADIO", "educ_priv", educ_privados)
df = add_indicator("RADIO", "educ_pub", educ_publicos)

In [9]:
df = add_indicator("FRAC", "hospitales", count_points(hospitales_shp, fracc_shp))
df = add_indicator("FRAC", "educ_priv", count_points(educ_privados_shp, fracc_shp))
df = add_indicator("FRAC", "educ_pub", count_points(educ_publicos_shp, fracc_shp))

In [10]:
df = add_indicator("DPTO", "hospitales", count_points(hospitales_shp, comuna_shp))
df = add_indicator("DPTO", "educ_priv", count_points(educ_privados_shp, comuna_shp))
df = add_indicator("DPTO", "educ_pub", count_points(educ_publicos_shp, comuna_shp))

## 3. Distancia media de los habs de un radio censal a puntos de transporte

In [11]:
def calculate_distance_indicators(df_indicators, id_field, division_shp_name, indicators):
    shapes_dict = {shape[0]: shape[1] for shape 
                   in iter_shp_as_shapely(get_division_path(division_shp_name))}
    
    for indic_name, indic_shp_name in indicators.iteritems():
        print "Calculating", indic_name, indic_shp_name
        
        multipoint = MultiPoint([shape[1].centroid for shape in 
                                 iter_shp_as_shapely(get_transport_shp_path(indic_shp_name))])

        def get_distance(id_shape):
            return shapes_dict[id_shape].centroid.distance(multipoint)
        
        df_indicators[indic_name] = map(get_distance, df_indicators[id_field])    

In [16]:
indicadores = {"d_subte": "subte-estaciones",
               "d_ffcc": "estaciones-ferrocarril",
               "d_metrobus": "metrobus-estaciones"}
indicators_radio = get_or_create_indicators_df("RADIO")
calculate_distance_indicators(indicators_radio, "CO_FRAC_RA", "radios_censo_2010", indicadores)

Calculating d_metrobus metrobus-estaciones
Calculating d_ffcc estaciones-ferrocarril
Calculating d_subte subte-estaciones
